In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

Importing ProductClassifier.csv webscraped from the website having ctaegory and ingredients of a particular product

In [2]:
df=pd.read_csv('/content/MamaEarthProductClassifier.csv')
df.head()

,Unnamed: 0,ProductName,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Category,Ingredients
0,0,Tea Tree Facewash for acne,NaN,NaN,NaN,NaN,NaN,Face Wash,Neem
1,1,Anti Hair Fall Kit for complete hair care,NaN,NaN,NaN,NaN,NaN,Hair Combo,Bhringraj Oil
2,2,"Argan Hair Mask with Argan, Avocado Oil, and M...",NaN,NaN,NaN,NaN,NaN,Hair Mask,Argan Oil
3,3,Ubtan Bamboo Sheet Mask with Turmeric & Saffro...,NaN,NaN,NaN,NaN,NaN,Face Mask,Turmeric
4,4,Niacinamide Bamboo Sheet Mask with Niacinamide...,NaN,NaN,NaN,NaN,NaN,Face Sheet Mask,Niacinamide


In [3]:
df.columns

Index(['Unnamed: 0', 'ProductName', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Category', 'Ingredients'],
      dtype='object')

In [4]:
df.drop(['Unnamed: 0','Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6'],axis=1,inplace=True)

In [5]:
df.head()

,ProductName,Category,Ingredients
0,Tea Tree Facewash for acne,Face Wash,Neem
1,Anti Hair Fall Kit for complete hair care,Hair Combo,Bhringraj Oil
2,"Argan Hair Mask with Argan, Avocado Oil, and M...",Hair Mask,Argan Oil
3,Ubtan Bamboo Sheet Mask with Turmeric & Saffro...,Face Mask,Turmeric
4,Niacinamide Bamboo Sheet Mask with Niacinamide...,Face Sheet Mask,Niacinamide


### 1. Classifying Categories 

Dropping Ingredients column and doing pre-processing

In [6]:
df1=df.drop(['Ingredients'],axis=1)

In [7]:
df1.head()

,ProductName,Category
0,Tea Tree Facewash for acne,Face Wash
1,Anti Hair Fall Kit for complete hair care,Hair Combo
2,"Argan Hair Mask with Argan, Avocado Oil, and M...",Hair Mask
3,Ubtan Bamboo Sheet Mask with Turmeric & Saffro...,Face Mask
4,Niacinamide Bamboo Sheet Mask with Niacinamide...,Face Sheet Mask


In [8]:
df1['Category'].unique()

array(['Face Wash', 'Hair Combo', 'Hair Mask', 'Face Mask',
       'Face Sheet Mask', 'Hair Serum', 'Hair Oil', 'Shampoo',
       'Face Combo', 'Body Lotion', 'Face Cream', 'Face Scrub',
       'Face Toner', 'Hair Gel', 'Face Serum', 'Conditioner',
       'Skin Combo', 'Body Wash', 'Face Gel', 'Body Scrub', 'Sanitizer'],
      dtype=object)

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
def product_name_cleaned(sample):
  # replacing everything with " " except alphabets
  sample=re.sub(pattern='[^a-zA-Z]',repl=' ',string=sample)
  # converting to lowercase
  sample=sample.lower()
  # splitting into a list of words
  sample_words=sample.split()
  # every word in list is accepted except stopwords
  sample_words=[word for word in sample_words if not word in set(stopwords.words('english'))]
  # stemming to root word
  result=[PorterStemmer().stem(word) for word in sample_words ]
  # creating a new string
  result=' '.join(result)
  return result

In [12]:
product_name_cleaned(df['ProductName'][0])

'tea tree facewash acn'

In [13]:
# Applying our function to whole dataset
df1['ProductName'] = df1['ProductName'].apply(product_name_cleaned)

In [14]:
# Creating features and labels
x=df1.ProductName
y=df1.Category
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [15]:
# Using Multinomial NaiveBayesClassifier using Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
nbc = Pipeline([('vectorized', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB()),
               ])
nbc.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vectorized',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [16]:
# Evaluating Model
from sklearn.metrics import classification_report, confusion_matrix
pred = nbc.predict(x_test)
print(classification_report(y_test,pred))

                 precision    recall  f1-score   support

    Body Lotion       1.00      1.00      1.00         2
     Body Scrub       0.00      0.00      0.00         1
      Body Wash       0.00      0.00      0.00         1
    Conditioner       0.00      0.00      0.00         1
     Face Combo       0.00      0.00      0.00         1
     Face Cream       1.00      0.20      0.33         5
      Face Mask       0.33      0.33      0.33         3
     Face Scrub       0.00      0.00      0.00         3
     Face Serum       0.00      0.00      0.00         1
Face Sheet Mask       1.00      0.50      0.67         2
     Face Toner       0.00      0.00      0.00         2
      Face Wash       0.18      1.00      0.30         3
     Hair Combo       0.36      1.00      0.53         4
      Hair Mask       0.00      0.00      0.00         2
       Hair Oil       0.00      0.00      0.00         3
     Hair Serum       0.00      0.00      0.00         2
        Shampoo       0.33    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# Testing input
test_product=['Onion Shampoo for Hair Growth & Hair Fall Control']
nbc.predict(test_product)

array(['Shampoo'], dtype='<U15')

### 2. Classifying Ingredients

We will be using the same dataset as above but will drop  Ctaregories column this time and do pre-processing

In [18]:
df2=df.drop(['Category'],axis=1)

In [19]:
df2.head()

,ProductName,Ingredients
0,Tea Tree Facewash for acne,Neem
1,Anti Hair Fall Kit for complete hair care,Bhringraj Oil
2,"Argan Hair Mask with Argan, Avocado Oil, and M...",Argan Oil
3,Ubtan Bamboo Sheet Mask with Turmeric & Saffro...,Turmeric
4,Niacinamide Bamboo Sheet Mask with Niacinamide...,Niacinamide


In [20]:
# Creating features and labels
x=df2.ProductName
y=df2.Ingredients
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [24]:
# Applying our function to whole dataset
df2['ProductName'] = df2['ProductName'].apply(product_name_cleaned)

In [25]:
# Using Multinomial NaiveBayesClassifier using Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
nbc = Pipeline([('vectorized', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB()),
               ])
nbc.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vectorized',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [26]:
# Evaluating Model
from sklearn.metrics import classification_report, confusion_matrix
pred = nbc.predict(x_test)
print(classification_report(y_test,pred))

                precision    recall  f1-score   support

     Aloe Vera       0.00      0.00      0.00         1
     Argan Oil       1.00      1.00      1.00         1
 Bhringraj Oil       1.00      1.00      1.00         1
      Charcoal       0.00      0.00      0.00         1
        Coffee       1.00      1.00      1.00         2
          Neem       0.00      0.00      0.00         2
   Niacinamide       0.00      0.00      0.00         1
     Onion Oil       0.00      0.00      0.00         1
Onion Seed Oil       0.50      1.00      0.67         4
     Red Onion       0.00      0.00      0.00         3
       Retinol       0.00      0.00      0.00         1
    Rose Water       0.00      0.00      0.00         1
       Saffron       0.00      0.00      0.00         1
  Tea Tree Oil       1.00      0.20      0.33         5
      Turmeric       0.43      0.60      0.50         5
     Vitamin C       0.43      1.00      0.60         9
  Walnut Beads       0.00      0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# Testing input
test_product=['Onion Shampoo for Hair Growth & Hair Fall Control']
nbc.predict(test_product)

array(['Onion Seed Oil'], dtype='<U20')